In [113]:
from LOTlib3.Grammar import Grammar
from LOTlib3.DefaultGrammars import DNF
from math import log
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood
from LOTlib3.DataAndObjects import FunctionData, Obj

from LOTlib3 import break_ctrlc
from LOTlib3.Miscellaneous import qq
from LOTlib3.TopN import TopN
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler

# Generative grammar

`grammar.add_rule( <NONTERMINAL>, <FUNCTION>, <ARGUMENTS>, <PROBABILITY>)`

Note: non-terminal arguments get passed as normal python arguments. E.g.: 

```is_color_(OBJECT, 'red') --> OBJECT.color == 'red'```

In [131]:


# Define a grammar object
grammar = Grammar(start='START')

grammar.add_rule('START', '', ['Q'], 1.00) # Quantifier

# The following lines are crucial for "recognizing OBJECTS provided in the data"
grammar.add_rule('Q', 'forall_', ['FUNCTION', 'SET'], 1.00)
grammar.add_rule('SET', 'S', None, 1.0)
grammar.add_rule('FUNCTION', 'lambda', ['DISJ'], 1.0, bv_type='X')

# Logical operations
grammar.add_rule('DISJ', 'or_', ['CONJ', 'DISJ'], 1.00)
grammar.add_rule('DISJ', 'False', None, 1.00)
grammar.add_rule('CONJ', 'and_', ['CONJ', 'P'], 1.00)
grammar.add_rule('CONJ', 'True', None, 1.00)

# Color
grammar.add_rule('P', 'is_color_',  ['X', '\'red\''],   1.00) 
grammar.add_rule('P', 'is_color_',  ['X', '\'green\''], 1.00)

# Shape
grammar.add_rule('P', 'is_shape_',  ['X', '\'square\''],   1.00) 
grammar.add_rule('P', 'is_shape_',  ['X', '\'triangle\''], 1.00)

# Size
grammar.add_rule('P', 'is_size_',  ['X', '\'big\''],   1.00) 
grammar.add_rule('P', 'is_size_',  ['X', '\'small\''], 1.00)

# Terminals
# grammar.add_rule('TRUE', 'True', None, 1.0)
# grammar.add_rule('FALSE', 'False', None, 1.0)

for _ in range(10):
    t = grammar.generate()
    print(f'{grammar.log_probability(t):.3f}', t)

-0.693 forall_(lambda y2: False, S)
-10.920 forall_(lambda y2: or_(and_(and_(and_(True, is_shape_(y2, 'triangle')), is_size_(y2, 'small')), is_size_(y2, 'big')), or_(True, False)), S)
-0.693 forall_(lambda y2: False, S)
-4.564 forall_(lambda y2: or_(and_(True, is_size_(y2, 'big')), False), S)
-0.693 forall_(lambda y2: False, S)
-2.079 forall_(lambda y2: or_(True, False), S)
-4.564 forall_(lambda y2: or_(and_(True, is_shape_(y2, 'square')), False), S)
-0.693 forall_(lambda y2: False, S)
-0.693 forall_(lambda y2: False, S)
-2.079 forall_(lambda y2: or_(True, False), S)


# Data

In [133]:
datum1 = FunctionData(
    input=[{Obj(color='red', shape='triangle', size='small')}], 
    output=True, 
    alpha=0.99
)
datum2 = FunctionData(
    input=[{Obj(color='green', shape='triangle', size='big')}], 
    output=False, 
    alpha=0.99
)
data = [datum1, datum2]

# Inference

In [ ]:
class MyHypothesis(BinaryLikelihood, LOTHypothesis):
    def __init__(self, grammar=grammar, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, display='lambda S: %s', **kwargs)

**Two hypotheses**

In [109]:
h1 = MyHypothesis()
h2 = MyHypothesis()
l1 = h1.compute_likelihood(data)
l2 = h2.compute_likelihood(data)
gt, lt = '>', '<'
print(
    f'• The hypotheses are: ',
    f'\t• {h1} (prior = {h1.compute_prior():.3f})',
    f'\t• {h2} (prior = {h2.compute_prior():.3f})',
    f'• Given:',
    f'\t• a positive example: {datum1}',
    f'\t• a negative example: {datum2}',
    f'• We reason that:',
    f'\t• the likelihood of h1 (={l1:.3f}) {gt if l1 > l2 else lt} h2 (={l2:.3f})',
    f'• Thus the posterior beliefs are:',
    f'\t• h1 = {h1.compute_posterior(data):.3f}',
    f'\t• h2 = {h2.compute_posterior(data):.3f}',
    sep='\n'
)

• The hypotheses are: 
	• lambda S: forall_(lambda y2: is_color_(y2, 'green'), S) (prior = -2.079)
	• lambda S: forall_(lambda y2: is_shape_(y2, 'square'), S) (prior = -2.079)
• Given:
	• a positive example: <{<OBJECT: color=red shape=triangle size=small >} -> True>
	• a negative example: <{<OBJECT: color=green shape=triangle size=small >} -> False>
• We reason that:
	• the likelihood of h1 (=-10.597) < h2 (=-5.303)
• Thus the posterior beliefs are:
	• h1 = -12.676
	• h2 = -7.383


**MCMC**

In [134]:
h0 = MyHypothesis()

top  = TopN(N=10)
thin = 1_000
for i, h in enumerate(break_ctrlc(MetropolisHastingsSampler(h0, data, steps=100_000))):
    top << h
    if i % thin == 0:
        print(f'{i}), post={h.posterior_score:.2f}, prior={h.prior:.2f}, lik={h.likelihood:.2f}, qq={qq(h)}')


print('=== WINNERS ===')
for i, h in enumerate(top):
    print(f'{i}), post={h.posterior_score:.2f}, prior={h.prior:.2f}, lik={h.likelihood:.2f}, qq={qq(h)}')

0), post=-11.54, prior=-6.24, lik=-5.30, qq="lambda S: forall_(lambda y2: or_(is_shape_(y2, 'triangle'), is_size_(y2, 'small')), S)"
1000), post=-10.41, prior=-10.40, lik=-0.01, qq="lambda S: forall_(lambda y2: and_(and_(is_size_(y2, 'small'), is_size_(y2, 'small')), is_color_(y2, 'red')), S)"
2000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: forall_(lambda y2: is_size_(y2, 'small'), S)"
3000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: forall_(lambda y2: is_color_(y2, 'red'), S)"
4000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: forall_(lambda y2: is_size_(y2, 'small'), S)"
5000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: forall_(lambda y2: is_color_(y2, 'red'), S)"
6000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: forall_(lambda y2: is_size_(y2, 'small'), S)"
7000), post=-6.25, prior=-6.24, lik=-0.01, qq="lambda S: forall_(lambda y2: and_(is_size_(y2, 'small'), is_shape_(y2, 'triangle')), S)"
8000), post=-2.09, prior=-2.08, lik=-0.01, qq="lambda S: